In [10]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import datetime
from tqdm import tqdm
from random import randrange

In [11]:
Dates_r= pd.date_range(start='1/1/2016',end='13/5/2021',freq='M')
Dates=[]
for i in Dates_r:
  Dates.append(str(i)[:4]+ str(i)[5:7])

In [12]:
urls=[]
for k in range(len(Dates)):
  url= 'http://www.estesparkweather.net/archive_reports.php?date='
  url += Dates[k]
  urls.append(url)

In [23]:
Latitude_coordinates=[]
Longitude_coordinates=[]
for url in urls:
  pages= requests.get(url)
  soup= BeautifulSoup(pages.content,'html.parser')
  rows = soup.find_all('tr')
  for tr in rows:
      cols = tr.find_all('td',attrs={'class':'data'})
      for td in cols:
        data=td.text
        Lat_value= re.compile(r'(\d\d):(\d\d):(\d\d\w)')
        Lat_coord= Lat_value.match(data)
        Long_value= re.compile(r'(\d\d\d):(\d\d):(\d\d\w)')
        Long_coord= Long_value.match(data)
        if Lat_coord != None:
          Latitude= ':'.join(Lat_coord.groups())
          Latitude_coordinates.append(Latitude)
        if Long_coord != None:
          Longitude= ':'.join(Long_coord.groups())
          Longitude_coordinates.append(Longitude)

In [24]:
coordinates=[]
for lat in Latitude_coordinates:
  for long in Longitude_coordinates:
    coordinates.append(lat + ' & '+ long)

In [41]:
Date_range=[]
data=[]
for url in urls:
  pages= requests.get(url)
  soup= BeautifulSoup(pages.content,'html.parser')
  table= soup.find_all('table')

  raw_data = [row.text.splitlines() for row in table]
  raw_data = raw_data[:-9]
  for i in range(len(raw_data)):
    raw_data[i] = raw_data[i][2:len(raw_data[i]):3]
    c = ['.'.join(re.findall('\d+',str(raw_data[i][j].split()[:5])))for j in range(len(raw_data[i]))]
    data.append(c)

    date= url.split('=')[-1]
    Date_range.append('-'.join([date[:4],date[4:6]]))

In [42]:
df= pd.DataFrame(columns=['Day','Average temperature','Average humidity','Average dewpoint','Average barometer','Average windspeed','Average gustspeed',
                          'Average direction','Rainfall for month','Rainfall for year','Maximum rain per minute','Maximum temperature',
                          'Minimum temperature','Maximum humidity','Minimum humidity','Maximum pressure','Minimum pressure','Maximum windspeed',
                          'Maximum gust speed','Maximum heat index'])

In [43]:
for i in range(len(data)):
  if len(data[i]) == 20:
    a_series= pd.Series(data[i],index=df.columns)
    df= df.append(a_series,ignore_index=True)

In [44]:
for i in range(len(data)):
  if len(data[i]) != 20:
    Date_range.remove(Date_range[i])
df['Date_range']= Date_range
df['Region']=coordinates[0]
first_column = df.pop('Region')
df.insert(1, 'Region', first_column)
df= df.set_index('Date_range')
df

,Day,Region,Average temperature,Average humidity,Average dewpoint,Average barometer,Average windspeed,Average gustspeed,Average direction,Rainfall for month,Rainfall for year,Maximum rain per minute,Maximum temperature,Minimum temperature,Maximum humidity,Minimum humidity,Maximum pressure,Minimum pressure,Maximum windspeed,Maximum gust speed,Maximum heat index
Date_range,,,,,,,,,,,,,,,,,,,,,
2016-01,1,40:23:00N & 105:29:00W,7.5,51,9.4,30.8,0.7,1.3,56,0.00,0.00,0.00,37.8,12.2,75,11,31.021,30.543,6.9,10.4,37.8
2016-01,2,40:23:00N & 105:29:00W,15.7,49,2.9,30.6,0.6,1.1,3,0.00,0.00,0.00,40.5,4.8,76,15,30.832,30.353,8.1,12.7,40.5
2016-01,3,40:23:00N & 105:29:00W,31.0,41,8.5,30.3,1.8,3.0,350,0.00,0.00,0.00,48.8,17.2,72,20,30.515,30.065,13.8,19.6,48.8
2016-01,4,40:23:00N & 105:29:00W,25.0,57,9.8,30.2,0.9,1.4,275,0.00,0.00,0.00,48.6,11.6,81,19,30.407,29.982,11.5,16.1,48.6
2016-01,5,40:23:00N & 105:29:00W,28.4,53,11.8,29.9,2.2,3.4,334,0.00,0.00,0.00,46.0,13.8,75,24,30.139,29.694,12.7,19.6,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04,26,40:23:00N & 105:29:00W,53.8,18,10.0,29.2,10.1,15.4,232,0.97,2.35,0.00,63.0,43.5,38,9,29.305,29.184,25.3,40.3,63.0
2021-04,27,40:23:00N & 105:29:00W,38.4,68,27.2,29.5,2.3,3.8,13,1.20,2.58,0.01,49.2,31.3,95,21,29.864,29.303,13.8,19.6,49.2
2021-04,28,40:23:00N & 105:29:00W,38.6,76,30.6,30.0,1.5,2.6,6,2.08,3.46,0.30,53.1,29.9,96,30,30.090,29.863,12.7,19.6,53.1


In [45]:
df.to_csv('weather_data.csv',index=True)